In [1]:
import os
import requests
import re
from bs4 import BeautifulSoup
import openpyxl

In [2]:
def hasdigit(url):
    return any(char.isdigit() for char in url)

In [3]:
def HavaAlphabet(url):
    return any(char.isalpha() for char in url)

In [4]:
def TaiwanTimesWebFilter(url):
    feature1 = url[:26]
#     print(feature1)
    feature2 = url[-4:]
    feature3 = url[28:40]
#     print(feature2)
    if feature1 != 'https://thetaiwantimes.com':
        return False
    elif hasdigit(feature2):
        return False
    elif '/' in feature3:
        return False
    else:
        return True

In [5]:
taiwantimes = 'https://thetaiwantimes.com/category/business'

In [6]:
saving_path = '.\\7_websites_output'
if not os.path.exists(saving_path):
    os.makedirs(saving_path)

In [7]:
TwT_homePage_head = ".cs-container"
TwT_homePage_list = ".archive-main"
TwT_NewsPage = ".entry-content"

In [8]:
def ft_TWT():
    prefix = 'https://thetaiwantimes.com/'
    r = requests.get(taiwantimes)

    if r.status_code == 200:  # print(r.status_code)
        soup = BeautifulSoup(r.text, 'html.parser')
        result1 = soup.select(TwT_homePage_head)
        result2 = soup.select(TwT_homePage_list)


    target_url_list = []

    for results in result1:
        class_a = results.find_all("a")
        for i in class_a:
            try:
                if TaiwanTimesWebFilter(i['href']) and (i['href'] not in target_url_list):
                    target_url_list.append(i['href'])
            except:
                pass

    for results in result2:
        class_a = results.find_all("a")
        for i in class_a:
            try:
                if TaiwanTimesWebFilter(i['href']) and (i['href'] not in target_url_list):
                    target_url_list.append(i['href'])
            except:
                pass

    workbook = openpyxl.Workbook()
    sheet = workbook.active
    ctr = 1

    for url in target_url_list[:]:
        r = requests.get(url)
        if r.status_code == 200:
            soup = BeautifulSoup(r.text, 'html.parser')
            result = soup.select(TwT_NewsPage)
            title_select = soup.find("h1")

            sheet['A'+str(ctr)] = title_select.text
            sheet['B'+str(ctr)] = url

            # print("-----------------------------------------------------------------------------------------------")
            # print(title_select.text)
            # print(url)
            content = []
            for results in result:
                paragraph = results.find_all("p")
                for i in paragraph:
                    content.append(i.text)

            if content == [] or len(content) < 5:
                raw_sentence = ''
                for i in result:
                    raw_sentence = i.text
                sentences = []
                raw_sentence_split = raw_sentence.split('\r')
                for i in raw_sentence_split:
                    sentences.append(i.strip())
                for i in sentences:
                    if i == '' or i[0]== '[':
                        pass
                    else:
                        content.append(i)

            if content == []:
                print("Can't crawler the web")
                print(url)
                sheet['C'+str(ctr)] = "Can't Crawler the Web"
                sheet['D'+str(ctr)] = ''
                ctr += 1
            else:
                for sentence in content:
                    sheet['D'+str(ctr)] = str(sentence)
    #                 print(sentence)
                    ctr += 1

    workbook.save(saving_path+'\\TaiwanTimes.xlsx')
    
    print('finish')

In [9]:
ft_TWT()

finish
